# Task 4: Conversational Interface - Chat with Llama 3 and Titan Premier LLMs

In this notebook, you build a chatbot using the llama3-8b-instruct and titan-text-premier Foundation Models (FMs) in Amazon Bedrock.

Conversational interfaces such as chatbots and virtual assistants can enhance the user experience for your customers. Chatbots use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. You can use chatbots in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. Users can access them through various channels such as websites, social media platforms, and messaging apps.

- **Chatbot (Basic)**, Zero Shot chatbot with a FM model
- **Chatbot using prompt**, template(LangChain) - Chatbot with some context provided in the prompt template
- **Chatbot with persona**, Chatbot with defined roles. i.e. Career Coach and Human interactions
- **Contextual-aware chatbot**, Passing in context through an external file by generating embeddings.

## LangChain framework for building Chatbot with Amazon Bedrock

In conversational interfaces such as chatbots, remembering previous interactions becomes highly important, both at a short-term and long-term level.

The LangChain framework provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular. Secondly, LangChain provides easy ways to incorporate these utilities into chains, allowing you to easily define and interact with different types of abstractions, which make powerful chatbots easy for you to build.

## Building a Chatbot with Context - Key Elements

The first process in building a context-aware chatbot is to generate embeddings for the context. Typically, you have an ingestion process which runs through your embedding model and generates the embeddings, which will then be stored in a vector store. In this notebook, you use the Titan Embeddings model for this. The second process is user request orchestration, interaction, invoking, and returning the results. This involves orchestrating the user request, interacting with the necessary models/components to gather information, invoking the chatbot to formulate a response, and then returning the chatbot's response back to the user.

## Task 4.1: Environment setup

In this task, you set up your environment.

In [1]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime')


In [45]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

In [46]:
# example of formatting. <|end_header_id|> is the end of message and is understandable if model is trained on that
instructions = [
    {"role": "system", "content": "You are an AI assistant."},
    {"role": "user", "content": "What is the capital of France?"},
]
print(format_instructions(instructions))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an AI assistant. <|eot_id|><|start_header_id|>user<|end_header_id|>
What is the capital of France? <|eot_id|><|start_header_id|>assistant<|end_header_id|>



## Task 4.2: Using chat history from LangChain to start the conversation

In this task, you enable the chatbot to carry conversational context across multiple interactions with users. Having a conversational memory is crucial for Chatbots to hold meaningful, coherent dialogues over time.

You implement conversational memory capabilities by building on top of LangChain's InMemoryChatMessageHistory class. This object stores the conversations between the user and the chatbot, and the history is available the chatbot agent so that it can leverage the context from a previous conversation.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The model outputs are non-deterministic.

In [53]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws import ChatBedrock

chat_model=ChatBedrock(
    # model_id="meta.llama3-8b-instruct-v1:0" , 
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_client)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


chain = prompt | chat_model | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
query="how are you?"
response=wrapped_chain.invoke({"input": query})
# Printing history to see the history being built out. 
print(history)
# For the rest of the conversation, the output will only include response

Human: how are you?
AI: I'm doing well, thanks for asking! As an AI language model, I don't have physical feelings or emotions, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


### New Questions

The model has responded with an initial message. Now, you ask it a few questions.

In [54]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
response=wrapped_chain.invoke({"input": format_instructions(instructions)})  # I tested without the format and it worked as well but history was spoiled
print(response)


Here are some tips to help you start a new garden:

1. Plan your garden: Decide what type of garden you want to create (vegetable, flower, etc.), consider the available space, light conditions, and your climate. This will help you choose the right plants.

2. Prepare the soil: Loosen the soil to a depth of 6-8 inches and add organic matter like compost or well-rotted manure to improve soil structure and fertility.

3. Choose the right plants: Select plants that are well-suited to your climate and growing conditions. Consider factors like sunlight, soil type, and water needs.

4. Arrange your plants: Lay out your plants in a way that is visually appealing and allows for proper growth and spacing.

5. Water and mulch: Water your plants regularly, especially during dry spells. Apply a 2-4 inch layer of mulch around your plants to retain moisture and suppress weeds.

6. Fertilize: Use a balanced, slow-release fertilizer to provide essential nutrients for your plants.

7. Weed and maintain:

In [55]:
print(history)

Human: how are you?
AI: I'm doing well, thanks for asking! As an AI language model, I don't have physical feelings or emotions, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?
Human: <|start_header_id|>user<|end_header_id|>
Give me a few tips on how to start a new garden. <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI: Here are some tips to help you start a new garden:

1. Plan your garden: Decide what type of garden you want to create (vegetable, flower, etc.), consider the available space, light conditions, and your climate. This will help you choose the right plants.

2. Prepare the soil: Loosen the soil to a depth of 6-8 inches and add organic matter like compost or well-rotted manure to improve soil structure and fertility.

3. Choose the right plants: Select plants that are well-suited to your climate and growing conditions. Consider factors like sunlight, soil type, and water needs.

4. Arrange